In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from ucimlrepo import fetch_ucirepo 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# fetch dataset 
phishing_websites = fetch_ucirepo(id=327) 
  
# data (as pandas dataframes) 
X = phishing_websites.data.features 
y = phishing_websites.data.targets

# Chia tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (8844, 30), Test size: (2211, 30)


In [64]:

from scipy.stats import randint, uniform

param_distributions = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 15)
}


In [67]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import randint, uniform

class RandomizedSearchCV_from_scratch:
    def __init__(self, estimator, param_distributions, n_iter=10, cv=5, random_state=None):
        self.estimator = estimator  # Mô hình cần tối ưu hóa
        self.param_distributions = param_distributions  # Phân phối tham số
        self.n_iter = n_iter  # Số lần thử nghiệm
        self.cv = cv  # Số fold cho cross-validation
        self.random_state = random_state  # Seed cho tái lập

        # Thiết lập seed nếu cần
        if self.random_state is not None:
            np.random.seed(self.random_state)

    def sample_params(self):
        """
        Lấy mẫu ngẫu nhiên các tham số từ phân phối đã cho.
        """
        sampled_params = {}
        for param, dist in self.param_distributions.items():
            if isinstance(dist, list):  # Nếu là danh sách, lấy ngẫu nhiên
                sampled_params[param] = np.random.choice(dist)
            else:  # Nếu là phân phối xác suất, lấy mẫu từ đó
                sampled_params[param] = dist.rvs()
        return sampled_params

    def cross_val_score(self, X, y, params):
        """
        Thực hiện cross-validation (StratifiedKFold) để tính điểm cho mô hình với tham số đã cho.
        """
        skf = StratifiedKFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)
        scores = []

        # Chạy CV
        for train_idx, valid_idx in skf.split(X, y):
            _X_train, _X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            _y_train, _y_valid = y.iloc[train_idx], y.iloc[valid_idx]

            # Huấn luyện mô hình với tham số đã chọn
            self.estimator.set_params(**params)
            self.estimator.fit(_X_train, _y_train)

            # Dự đoán và tính điểm accuracy
            y_pred = self.estimator.predict(_X_valid)
            score = accuracy_score(_y_valid, y_pred)
            scores.append(score)

        return np.mean(scores)  # Trả về điểm số trung bình của tất cả các fold

    def fit(self, X, y):
        """
        Tiến hành tìm kiếm ngẫu nhiên các tham số và huấn luyện mô hình với Cross-Validation.
        """
        best_score = -np.inf
        best_params = None
        results = []

        for i in range(self.n_iter):
            # Lấy mẫu các tham số
            sampled_params = self.sample_params()

            # Tính điểm cho tham số hiện tại với Cross-Validation
            score = self.cross_val_score(X, y, sampled_params)

            results.append({'params': sampled_params, 'score': score})

            # Cập nhật nếu điểm số hiện tại là tốt nhất
            if score > best_score:
                best_score = score
                best_params = sampled_params

        # Lưu kết quả và trả về tham số tốt nhất
        self.results_ = results
        self.best_score_ = best_score
        self.best_params_ = best_params

    def get_results(self):
        """
        Trả về tất cả kết quả tìm kiếm.
        """
        return self.results_

In [71]:
rf = RandomForestClassifier(random_state=42)

In [72]:

search = RandomizedSearchCV_from_scratch(rf, param_distributions, n_iter=10, cv=5, random_state=42)
search.fit(X_train, y_train)

KeyError: "None of [Index([   1,    2,    3,    4,    5,    6,    7,    8,   10,   12,\n       ...\n       8832, 8833, 8834, 8835, 8838, 8839, 8840, 8841, 8842, 8843],\n      dtype='int32', length=7075)] are in the [columns]"